# Installing FlightRadarAPI

In [ ]:
pip install FlightRadarAPI

**Import the class FlightRadar24API and create an instance of it.**

In [1]:
#Standard import 
import pandas as pd
import numpy as np
from FlightRadar24 import FlightRadar24API
fr_api = FlightRadar24API()

In [2]:
flights = fr_api.get_flights()

flights_data = []

for flight_obj in flights:
    # Obtain the model for the current flight's aircraft code
    flight_data = {
        'Aircraft Code': flight_obj.aircraft_code,
        'Airline IATA': flight_obj.airline_iata,
        'Airline ICAO': flight_obj.airline_icao,
        'Altitude': flight_obj.altitude,
        'Callsign': flight_obj.callsign,
        'Destination Airport IATA': flight_obj.destination_airport_iata,
        'Ground Speed': flight_obj.ground_speed,
        'Heading': flight_obj.heading,
        'ICAO 24bit': flight_obj.icao_24bit,
        'Flight ID': flight_obj.id,
        'Latitude': flight_obj.latitude,
        'Longitude': flight_obj.longitude,
        'Flight Number': flight_obj.number,
        'On Ground': flight_obj.on_ground,
        'Origin Airport IATA': flight_obj.origin_airport_iata,
        'Registration': flight_obj.registration,
        'Squawk': flight_obj.squawk,
        'Time': flight_obj.time,
        'Vertical Speed': flight_obj.vertical_speed,

    }

    flights_data.append(flight_data)

df_flights = pd.DataFrame(flights_data)


In [3]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.airportcodes.us/us-airports.htm'

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    airport_codes_table = soup.find('table')
    usa_airport_codes = []

    if airport_codes_table:
        rows = airport_codes_table.find_all('tr')

        for row in rows:
            cols = row.find_all('td')
            if cols:
                usa_airport_code = cols[0].text.strip()

                if usa_airport_code.isupper() and len(usa_airport_code) == 3:
                    usa_airport_codes.append(usa_airport_code)

        #print(airport_codes)

else:
    print("Failed to retrieve the webpage")


In [4]:
df_flights_usa = df_flights[(df_flights['Origin Airport IATA'].isin(usa_airport_codes)) |
                            (df_flights['Destination Airport IATA'].isin(usa_airport_codes))]

In [5]:
df_flights_usa.shape

(623, 19)

In [6]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_airports_in_Brazil'

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the table containing the airport data
    table = soup.find('table', class_='wikitable sortable')

    # Check if the table is found
    brazil_airport_code= []
    if table:
        # Iterate over rows in the table
        for row in table.find_all('tr')[1:]:  # Start from index 1 to skip header row
            # Extract columns from the row
            columns = row.find_all('td')
            if columns:  # Ensure there are columns
                # Extract the IATA code (assuming it's the first column)
                iata_code = columns[3].text.strip()
                brazil_airport_code.append(iata_code)
        #print(brazil_airport_code)
    else:
        print("Table not found on the page.")
else:
    print("Failed to fetch the page.")



In [7]:

#brz_airport_codes = brazil_airport_code

# Filter for flights originating from or destined for Brazil
df_flights_brazil = df_flights[(df_flights['Origin Airport IATA'].isin(brazil_airport_code)) |
                               (df_flights['Destination Airport IATA'].isin(brazil_airport_code))]



In [8]:
df_flights_brazil

,Aircraft Code,Airline IATA,Airline ICAO,Altitude,Callsign,Destination Airport IATA,Ground Speed,Heading,ICAO 24bit,Flight ID,Latitude,Longitude,Flight Number,On Ground,Origin Airport IATA,Registration,Squawk,Time,Vertical Speed
1,BALL,,,65700,HBAL675,,15,116,A2609D,3415c436,39.9601,-90.5956,,0,,N252TH,,1708629554,128
17,B77L,EK,UAE,37000,UAE248,DXB,520,32,896175,34178b51,15.9412,53.9043,EK248,0,GIG,A6-EWC,,1708629553,0
29,B77W,QR,QTR,36000,QTR773,GRU,489,240,06A053,3417a431,-21.6727,-41.9875,QR773,0,DOH,A7-BAB,,1708629553,0
45,A388,EK,UAE,40000,UAE261,GRU,482,247,896476,3417a9d4,-20.6622,-39.1123,EK261,0,DXB,A6-EUL,,1708629554,0
60,A359,TK,THY,40000,THY15,GRU,485,236,4BB0E1,3417b132,-20.6061,-40.6796,TK15,0,IST,TC-LGA,,1708629554,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,B789,LA,LAN,37000,TAM8195,GRU,473,152,E80470,3418c0c6,-3.4143,-53.7213,LA8195,0,MIA,CC-BGU,,1708629553,64
1463,B762,TE,IGA,41000,IGA713,,542,80,48B204,3418c1c1,22.7748,95.0009,TE713,0,DIA,SP-MRE,,1708629553,0
1479,GLF6,,,43000,GLF6,,437,249,,3418c2b5,38.4732,-110.5041,,0,,,,1708629555,-64
1484,G280,,,0,G280,,0,177,,3418c360,30.1883,-97.6620,,1,,,,1708629554,0


In [24]:
df_flights_brazil.loc[0]

Aircraft Code                     GLID
Airline IATA                          
Airline ICAO                          
Altitude                          2240
Callsign                         STM32
Destination Airport IATA              
Ground Speed                         0
Heading                              0
ICAO 24bit                    10395004
Flight ID                     340a771f
Latitude                      -33.3779
Longitude                     -70.5793
Flight Number                         
On Ground                            0
Origin Airport IATA                   
Registration                     STM32
Squawk                                
Time                        1708592872
Vertical Speed                       0
Name: 0, dtype: object

In [32]:
df_flights_brazil[df_flights_brazil.loc[:,'Aircraft Code'] == 'GLEX']

,Aircraft Code,Airline IATA,Airline ICAO,Altitude,Callsign,Destination Airport IATA,Ground Speed,Heading,ICAO 24bit,Flight ID,Latitude,Longitude,Flight Number,On Ground,Origin Airport IATA,Registration,Squawk,Time,Vertical Speed
99,GLEX,,,30325,GLEX,,490,164,,3416b5ca,48.5976,6.4287,,0,,,,1708592888,-1024
161,GLEX,KE,KAL,43000,KAL2443,,390,267,71C230,3416dc01,19.8222,73.7614,KE2443,0,GMP,HL8230,,1708592887,0
192,GLEX,,,0,GLEX,,8,56,,3416ee2a,14.7918,-92.3695,,1,,,,1708592886,0
726,GLEX,,,45000,GLEX,,536,108,,34176b7b,22.9720,49.4052,,0,,,,1708592888,-64
855,GLEX,,,43000,GLEX,,411,277,,34177b6b,-32.2818,125.6565,,0,,,,1708592885,-64
1316,GLEX,,,43000,GLEX,,500,246,,34179fcd,63.5198,-68.8541,,0,,,,1708592884,64


In [9]:
df_flights_brazil.describe()

,Altitude,Ground Speed,Heading,Latitude,Longitude,On Ground,Time,Vertical Speed
count,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,1.420000e+02,142.000000
mean,27292.669014,338.014085,199.042254,17.339890,-35.832261,0.049296,1.708630e+09,-81.577465
std,16130.059868,170.416219,85.743380,27.150180,62.252227,0.217251,1.109992e+02,474.920872
min,0.000000,0.000000,1.000000,-36.950000,-149.996900,0.000000,1.708628e+09,-2368.000000
25%,12125.000000,168.500000,137.500000,-8.061275,-76.420700,0.000000,1.708630e+09,-64.000000
50%,34462.500000,422.500000,208.000000,25.067900,-47.116300,0.000000,1.708630e+09,0.000000
75%,40000.000000,465.750000,253.000000,38.740500,4.347075,0.000000,1.708630e+09,0.000000
max,65700.000000,572.000000,356.000000,64.686300,158.641600,1.000000,1.708630e+09,1280.000000


In [14]:
aircraftModel = pd.read_excel('/Users/apple/Desktop/Spring 2024/Big_Data_Development/Project/Aircraft_TypeDesignators(ICAO).xlsx')

In [23]:
aircraftModel

,Manufacturer,Model,Type Disignator,Description,Engine Type,Engine Count,WTC
0,ANTONOV,An-72,AN72,LandPlane,Jet,NaN,NaN
1,ANTONOV,An-74-200,AN72,LandPlane,Jet,2.0,M
2,RUTAN,151 Ares,ARES,LandPlane,Jet,2.0,M
3,SCALED,151 Ares,ARES,LandPlane,Jet,2.0,M
4,GULFSTREAM AEROSPACE,Gulfstream G100,ASTR,LandPlane,Jet,1.0,L
...,...,...,...,...,...,...,...
1196,YAKOVLEV,Yak-40,YK40,LandPlane,Jet,1.0,L
1197,YAKOVLEV,Yak-42,YK42,LandPlane,Jet,3.0,M
1198,YAKOVLEV,Yak-142,YK42,LandPlane,Jet,3.0,M
1199,SOKO-CNIAR,J-22 Orao,YURO,LandPlane,Jet,3.0,M


In [49]:
jet_models = aircraftModel['Type Disignator'].unique()

In [50]:
len(jet_models)

386

In [51]:
jet_modelcount = aircraftModel['Type Disignator'].value_counts()

In [52]:
jet_modelcount

Type Disignator
E145    23
F5      20
F18H    18
SBR1    17
LJ35    16
        ..
CKUO     1
F5SA     1
MG44     1
F2TH     1
CL30     1
Name: count, Length: 386, dtype: int64

In [56]:
df_flights_brazil[df_flights_brazil['Aircraft Code'].isin(jet_models)]

,Aircraft Code,Airline IATA,Airline ICAO,Altitude,Callsign,Destination Airport IATA,Ground Speed,Heading,ICAO 24bit,Flight ID,Latitude,Longitude,Flight Number,On Ground,Origin Airport IATA,Registration,Squawk,Time,Vertical Speed
17,B77L,EK,UAE,37000,UAE248,DXB,520,32,896175,34178b51,15.9412,53.9043,EK248,0,GIG,A6-EWC,,1708629553,0
29,B77W,QR,QTR,36000,QTR773,GRU,489,240,06A053,3417a431,-21.6727,-41.9875,QR773,0,DOH,A7-BAB,,1708629553,0
156,F2TH,,,43000,FWWJU,,400,228,3806DA,3417eb9c,37.5363,-88.3282,,0,,F-WWJU,,1708629552,128
230,GL7T,,,47650,GL7T,,464,208,,3418117c,-24.4344,120.4361,,0,,,,1708629556,512
231,E195,,AZU,33000,,,269,109,E48B9E,3418117d,-5.7793,-40.8911,,0,,PR-AXF,,1708629549,1024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,B789,LA,LAN,37000,TAM8195,GRU,473,152,E80470,3418c0c6,-3.4143,-53.7213,LA8195,0,MIA,CC-BGU,,1708629553,64
1463,B762,TE,IGA,41000,IGA713,,542,80,48B204,3418c1c1,22.7748,95.0009,TE713,0,DIA,SP-MRE,,1708629553,0
1479,GLF6,,,43000,GLF6,,437,249,,3418c2b5,38.4732,-110.5041,,0,,,,1708629555,-64
1484,G280,,,0,G280,,0,177,,3418c360,30.1883,-97.6620,,1,,,,1708629554,0


In [57]:
df_flights_brazil.value_counts()

Aircraft Code  Airline IATA  Airline ICAO  Altitude  Callsign  Destination Airport IATA  Ground Speed  Heading  ICAO 24bit  Flight ID  Latitude  Longitude  Flight Number  On Ground  Origin Airport IATA  Registration  Squawk  Time        Vertical Speed
                                           3100      SDW3328                             65            57                   3418ba24    39.1535  -96.8264                  0                                                     1708629557  -64               1
GL5T                                       43000     GL5T                                423           260                  3418c3c2   -23.3444   7.6252                   0                                                     1708629549   0                1
F2TH                                       43000     F2TH                                372           247                  3418bb95    55.5134  -52.0896                  0                                                     170862954